# Proton propagation using CRPropa3

Gabriel de Azeredo, April 2024

Proton propagation through an expanding Universe, filled by CMB. Source at 100 Mpc.

Electron pair production and pion production is take into account as interaction with photon background.

This script generate earth's measured spectrum of a source emitting 10000 protons in a power law. 

In [1]:
#    Module import

from crpropa import *

In [2]:
#    Simulation general setup

events = 10000
source_distance = 200 * Mpc

filename1 = 'sim/05_sim1D_PowerLawSource_100Mpc_Protons.txt'
filename2 = 'sim/05_sim1D_PowerLawSource_100Mpc_Photons.txt'
filename3 = 'sim/05_sim1D_PowerLawSource_100Mpc_Elec.txt'

photons = True
neutrinos = False
electrons = True

#    Photon field setup

cmb = CMB()

In [3]:
#    Monochromatic source setup

position = SourcePosition(source_distance)
direction = SourceDirection(Vector3d(-1, 0, 0)) 
redshift = SourceRedshift1D() # Add a redshift
energySpectrum = SourcePowerLawSpectrum(1 * EeV, 1e3 * EeV, -1)
particleType = SourceParticleType(nucleusId(1, 1)) # Emitting protons
source = Source()

source.add(position)
source.add(redshift) 
source.add(direction)
source.add(energySpectrum)
source.add(particleType)

In [4]:
#    Observer setup

obs1 = Observer() # proton output
obs1.add(Observer1D())
obs1.add(ObserverPhotonVeto()) # we don't want photons here
obs1.add(ObserverElectronVeto()) # we don't want electrons
out1 = TextOutput(filename1, Output.Event1D)
out1.setEnergyScale(eV)
out1.enable(Output.WeightColumn)
out1.disable(Output.CandidateTagColumn)
obs1.onDetection(out1)

In [5]:
obs2 = Observer() # photon output
obs2.add(Observer1D())
# obs2.add(ObserverDetectAll()) # stores the photons at creation without propagating them
obs2.add(ObserverElectronVeto())
obs2.add(ObserverNucleusVeto()) # we don't want nuclei here
out2 = TextOutput(filename2, Output.Event1D)
out2.setEnergyScale(eV)


out2.enable(Output.WeightColumn)
obs2.onDetection(out2)
out2.disable(Output.CandidateTagColumn)

In [6]:
obs3 = Observer() # electron output
obs3.add(Observer1D())
# obs3.add(ObserverDetectAll()) # stores the photons at creation without propagating them
obs3.add(ObserverPhotonVeto()) # we don't want photons
obs3.add(ObserverNucleusVeto()) # we don't want nuclei here
out3 = TextOutput(filename3, Output.Event1D)
out3.setEnergyScale(eV)
out3.enable(Output.WeightColumn)
out3.disable(Output.CandidateTagColumn)

# enables the necessary columns to be compatible with the DINT and EleCa propagation
# out2.enable(Output.CreatedIdColumn)
# out2.enable(Output.CreatedEnergyColumn)
# out2.enable(Output.CreatedPositionColumn)
obs3.onDetection(out3)

In [7]:
#    Energy loss processes 

processes = []

#    Adiabatic energy loss

setCosmologyParameters(0.673, 1) 

z = Redshift()
processes.append(z)

#    Electron pair production

eppCMB = ElectronPairProduction(cmb, True)
processes.append(eppCMB)

#    Photo pion production

pppCMB = PhotoPionProduction(cmb, False, False, False)
processes.append(pppCMB)





In [8]:
#    Defining propagator

propagator = SimplePropagation(0.1 * kpc, 1 * Mpc)

In [9]:
#    Breaking conditions

max_trajectory = MaximumTrajectoryLength(500 * Mpc)

In [10]:
#    Assemble simulation modules

sim = ModuleList()

sim.add(obs1)  # Observer
sim.add(obs2) 
sim.add(obs3) 
[sim.add(process) for process in processes] # Energy loss 
sim.add(propagator) # Propagator
sim.add(max_trajectory) # Break conditions

sim.add(EMPairProduction(cmb, electrons))
sim.add(EMDoublePairProduction(cmb, electrons))
sim.add(EMInverseComptonScattering(cmb, photons))
sim.add(EMTripletPairProduction(cmb, electrons))

sim.setShowProgress(True)

In [ ]:
#    Simulation run

sim.run(source, events)
output.close()

crpropa::ModuleList: Number of Threads: 12
Run ModuleList


In [ ]:
comovingDistance2Redshift(source_distance)